## Linear Regression

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats # pip install scipy
from scipy.stats import skew

plt.rcParams["figure.figsize"] = (10, 6)

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("25Wafer_10LayerVariation_SNR30db_train.csv", header=None, skiprows=4); df.head()

,0,1,2,3,4,5,6,7,8,9,...,592,593,594,595,596,597,598,599,600,601
0,1.04360,0.77653,0.84672,0.83199,1.52820,1.28200,1.03750,1.28180,0.98114,0.92108,...,0.55489,0.55656,0.50621,0.40879,0.50317,0.59060,0.41935,0.50326,0.51321,1500
1,1.06020,0.71712,0.90560,1.14410,1.07980,0.94290,0.95501,1.06060,1.23440,1.06240,...,0.49716,0.57034,0.55974,0.57759,0.43101,0.46000,0.52845,0.55909,0.46235,1500
2,0.05587,0.77704,0.75296,0.54951,0.84628,0.63615,0.93858,0.71361,0.92573,0.88881,...,0.35918,0.51248,0.45673,0.37929,0.50958,0.43262,0.53272,0.52882,0.38007,1500
3,0.57876,0.75194,0.77168,0.49729,0.95006,0.88738,0.85901,0.66705,1.15360,0.79081,...,0.47656,0.57479,0.53811,0.58468,0.52504,0.42394,0.48927,0.59442,0.50571,1500
4,0.63567,0.91243,0.64593,0.94196,0.85344,0.79872,0.91289,0.92319,1.01480,1.00150,...,0.58752,0.55076,0.51340,0.59283,0.48574,0.55767,0.49496,0.49007,0.44945,1500


In [3]:
df.shape

(98250, 602)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98250 entries, 0 to 98249
Columns: 602 entries, 0 to 601
dtypes: float64(601), int64(1)
memory usage: 451.3 MB


In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
0,98250.0,0.788568,0.167248,0.019069,0.675242,0.789490,0.901560,1.49080
1,98250.0,0.822137,0.164302,0.105110,0.711950,0.822915,0.933758,1.54560
2,98250.0,0.854240,0.161016,0.189840,0.746313,0.855030,0.962997,1.51410
3,98250.0,0.888209,0.157688,0.219410,0.782000,0.888180,0.994870,1.55950
4,98250.0,0.919420,0.154552,0.161590,0.815210,0.919320,1.024900,1.57490
...,...,...,...,...,...,...,...,...
597,98250.0,0.480588,0.067861,0.198950,0.433990,0.478090,0.525090,0.80875
598,98250.0,0.480800,0.068277,0.149530,0.433620,0.478840,0.525390,0.79680
599,98250.0,0.481497,0.069091,0.216870,0.434100,0.479000,0.526540,0.81020
600,98250.0,0.481729,0.069508,0.196560,0.434200,0.479235,0.526977,0.81041


In [6]:
df.isnull().count().sort_values(ascending=True)  #Not Null

0      98250
397    98250
398    98250
399    98250
400    98250
       ...  
202    98250
203    98250
204    98250
206    98250
601    98250
Length: 602, dtype: int64

In [7]:
df[601]

0        1500
1        1500
2        1500
3        1500
4        1500
         ... 
98245     200
98246     200
98247     200
98248     200
98249     200
Name: 601, Length: 98250, dtype: int64

In [8]:
corrs = df.corrwith(df[601])

In [9]:
pd.DataFrame(corrs).sample(10)

,0
462,0.707844
135,-0.092565
97,0.150623
267,0.377968
482,0.624555
4,0.067948
88,0.117132
298,0.739907
143,-0.199890
28,0.084586


In [10]:
pd.DataFrame(corrs).sort_values(by=0,ascending=False).head(20)

,0
601,1.000000
361,0.897987
354,0.897772
357,0.897673
360,0.897454
362,0.897449
356,0.897372
355,0.897345
359,0.897278
364,0.897200


In [11]:
pd.DataFrame(corrs).sort_values(by=0,ascending=False).sample(20)

,0
474,0.660953
296,0.726343
387,0.880496
217,-0.459233
271,0.439995
194,-0.641811
15,0.108342
137,-0.120130
417,0.834520
477,0.647570


In [12]:
df.isnull().sum().unique()

array([0], dtype=int64)

In [13]:
skew_list = []

for i in df.drop(columns = 601).columns:
    print(i, skew(df[i]))
    skew_list.append(skew(df[i]))

0 -0.02212470016242096
1 -0.03061682925248978
2 -0.031046409179058074
3 -0.014466791462917327
4 -0.014402603709737343
5 0.006096296526942419
6 -0.016555512399258634
7 -0.014608898094469493
8 -0.008359771773228773
9 0.006864029273465999
10 0.0029417169175403332
11 0.01745134378642265
12 -0.0035228276286788603
13 0.018277440010844172
14 0.0025576608584379447
15 0.010355538288733179
16 0.015911902932684328
17 0.023596453482393105
18 0.024756729877319947
19 0.024244438981413916
20 0.020825604127209858
21 0.01839236826923308
22 0.014019535359107455
23 0.0335315107629706
24 0.03223670328394589
25 0.019529385762513764
26 0.017162997769440316
27 0.03837823199487552
28 0.015844552172406316
29 0.019102129567944118
30 0.0251867652476731
31 0.02819878757425152
32 0.03783101089819568
33 0.03537047698999727
34 0.032872687874980946
35 0.03892389731411944
36 0.028060296844276453
37 0.03952078827966955
38 0.022018959633418326
39 0.0360373445149037
40 0.04061293611663603
41 0.03328058813837233
42 0.0440

351 0.01285314137927924
352 0.01918892909624514
353 0.024763533749633405
354 0.032178564043322117
355 0.04521109557943648
356 0.05337112943768541
357 0.059627922916808206
358 0.0649478800707406
359 0.07072637954497857
360 0.07745785820799928
361 0.08904876050839142
362 0.0923925451335634
363 0.0954172426557369
364 0.1035015208967325
365 0.11932860322026365
366 0.1234374509235997
367 0.13148261356318292
368 0.13781752187623852
369 0.14609602171614142
370 0.14820248845315045
371 0.1583508889394008
372 0.1598656966502696
373 0.16847356901425034
374 0.18000582335452373
375 0.18656957574285765
376 0.19385049296427895
377 0.1949882297397933
378 0.20163215211689048
379 0.20977125014791342
380 0.2160311519968086
381 0.22575471768160305
382 0.23190654366689373
383 0.23634388686368088
384 0.24278643317468362
385 0.24706336476009463
386 0.25396013301694814
387 0.25903667727950186
388 0.26867645782706706
389 0.2684579238674715
390 0.2791219909447027
391 0.2901537273702682
392 0.2882135158969352
39

In [14]:
pd.DataFrame(skew_list).sort_values(by=0,ascending=False).head(20)  #Normal dışı dağılım gösteren yok

,0
466,0.523210
468,0.522417
475,0.520074
464,0.519593
470,0.518626
458,0.518420
471,0.518416
469,0.518085
465,0.517969
459,0.517647


In [15]:
liste = []
j = 0

for i in list(range(25)):
    j = j+1
    for i in list(range(3930)):
        liste.append(j)
        
print(liste)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [16]:
df["wafer_id"] = liste; df.head(40)

,0,1,2,3,4,5,6,7,8,9,...,593,594,595,596,597,598,599,600,601,wafer_id
0,1.04360,0.77653,0.84672,0.83199,1.52820,1.28200,1.03750,1.28180,0.98114,0.92108,...,0.55656,0.50621,0.40879,0.50317,0.59060,0.41935,0.50326,0.51321,1500,1
1,1.06020,0.71712,0.90560,1.14410,1.07980,0.94290,0.95501,1.06060,1.23440,1.06240,...,0.57034,0.55974,0.57759,0.43101,0.46000,0.52845,0.55909,0.46235,1500,1
2,0.05587,0.77704,0.75296,0.54951,0.84628,0.63615,0.93858,0.71361,0.92573,0.88881,...,0.51248,0.45673,0.37929,0.50958,0.43262,0.53272,0.52882,0.38007,1500,1
3,0.57876,0.75194,0.77168,0.49729,0.95006,0.88738,0.85901,0.66705,1.15360,0.79081,...,0.57479,0.53811,0.58468,0.52504,0.42394,0.48927,0.59442,0.50571,1500,1
4,0.63567,0.91243,0.64593,0.94196,0.85344,0.79872,0.91289,0.92319,1.01480,1.00150,...,0.55076,0.51340,0.59283,0.48574,0.55767,0.49496,0.49007,0.44945,1500,1
5,0.94406,0.72647,0.87055,0.96330,0.87663,0.88548,0.92800,0.81987,0.94160,0.96096,...,0.59353,0.60213,0.58162,0.48609,0.60251,0.56271,0.46354,0.52453,1500,1
6,0.69759,0.62444,0.79454,0.83541,0.96774,0.84737,0.96561,1.14640,1.01360,0.96519,...,0.56962,0.51782,0.62391,0.42109,0.64563,0.50753,0.42562,0.47719,1500,1
7,0.69703,0.76967,0.55524,0.94511,0.84725,0.81649,0.57679,0.79763,1.06960,0.88621,...,0.54877,0.43220,0.50256,0.46862,0.55368,0.58483,0.44164,0.47083,1500,1
8,0.57166,0.74954,1.15830,1.01500,0.73082,0.88985,1.01280,0.89285,1.18390,1.24540,...,0.56145,0.55672,0.58980,0.53779,0.40885,0.48903,0.53385,0.64637,1500,1
9,0.73906,0.90407,0.65007,0.93111,1.23210,0.83828,1.06580,0.89957,1.12360,0.96680,...,0.52346,0.57134,0.53884,0.51515,0.51963,0.57613,0.39393,0.56475,1500,1


In [17]:
pd.Series(df[601].groupby(df["wafer_id"]))   #wafer id kontrolü için. Tüm deneylerin 1500 ile başlaması gerek. Hata yok

0     (1, [1500, 1500, 1500, 1500, 1500, 1500, 1500,...
1     (2, [1500, 1500, 1500, 1500, 1500, 1500, 1500,...
2     (3, [1500, 1500, 1500, 1500, 1500, 1500, 1500,...
3     (4, [1500, 1500, 1500, 1500, 1500, 1500, 1500,...
4     (5, [1500, 1500, 1500, 1500, 1500, 1500, 1500,...
5     (6, [1500, 1500, 1500, 1500, 1500, 1500, 1500,...
6     (7, [1500, 1500, 1500, 1500, 1500, 1500, 1500,...
7     (8, [1500, 1500, 1500, 1500, 1500, 1500, 1500,...
8     (9, [1500, 1500, 1500, 1500, 1500, 1500, 1500,...
9     (10, [1500, 1500, 1500, 1500, 1500, 1500, 1500...
10    (11, [1500, 1500, 1500, 1500, 1500, 1500, 1500...
11    (12, [1500, 1500, 1500, 1500, 1500, 1500, 1500...
12    (13, [1500, 1500, 1500, 1500, 1500, 1500, 1500...
13    (14, [1500, 1500, 1500, 1500, 1500, 1500, 1500...
14    (15, [1500, 1500, 1500, 1500, 1500, 1500, 1500...
15    (16, [1500, 1500, 1500, 1500, 1500, 1500, 1500...
16    (17, [1500, 1500, 1500, 1500, 1500, 1500, 1500...
17    (18, [1500, 1500, 1500, 1500, 1500, 1500, 

In [18]:
df_column = []
df_index = []
for i in np.arange(0,98280,30):        #30'ar deney var, ortalamalarını alıp tek değere düşüreceğiz
    df_index.append(df_column)         
    df_column = []                  #601 column için ort hesaplayıp bunu yukardaki listeye ekledikten sonra listeyi boşaltıyoruz
    for j in df.columns:
        df_column.append(df.loc[i:(i+29),j].mean())      #Her bir column için ayrı ayrı 30 deneyin ortalamasını alıp listeye ekliyor


In [19]:
df_mean = pd.DataFrame(df_index); df_mean.head(10)    #yukarda oluşan listeyi dataframe'e çevirdik. başta boş liste olduğu için ilk satır boş kaldı.

,0,1,2,3,4,5,6,7,8,9,...,593,594,595,596,597,598,599,600,601,602
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.698106,0.763157,0.776748,0.832544,0.914430,0.916932,0.944375,0.934293,1.009005,1.018605,...,0.540553,0.521360,0.531954,0.501646,0.515534,0.534039,0.539064,0.525850,1500.0,1.0
2,0.729856,0.751451,0.827925,0.801372,0.852115,0.927645,0.951585,0.984030,1.012301,1.054866,...,0.511890,0.507746,0.508945,0.523448,0.501185,0.540460,0.511928,0.526840,1490.0,1.0
3,0.669586,0.733506,0.868904,0.841791,0.832750,0.914250,0.933147,1.008497,1.028314,1.002629,...,0.541548,0.536088,0.520284,0.510242,0.509127,0.529585,0.545440,0.525062,1480.0,1.0
4,0.762774,0.802765,0.803625,0.847220,0.929512,0.954482,0.998781,1.015917,1.023009,1.077459,...,0.516289,0.501284,0.505742,0.540932,0.528400,0.521195,0.539930,0.517142,1470.0,1.0
5,0.714084,0.810382,0.775471,0.806111,0.873733,0.946925,0.942013,0.979679,1.000049,1.069794,...,0.513057,0.535731,0.515139,0.499030,0.524250,0.540792,0.515419,0.521071,1460.0,1.0
6,0.734868,0.781009,0.799552,0.850824,0.908510,0.928054,0.971433,1.024524,1.027336,1.011656,...,0.510011,0.537002,0.510141,0.502846,0.499722,0.521965,0.517660,0.524345,1450.0,1.0
7,0.731995,0.771950,0.772438,0.881989,0.932348,0.957673,0.981788,0.987127,1.048777,1.094625,...,0.530991,0.520688,0.535340,0.522624,0.503805,0.489862,0.496392,0.526878,1440.0,1.0
8,0.781571,0.813475,0.852045,0.936589,0.916573,0.972601,0.993279,1.058264,1.077179,1.095678,...,0.536279,0.521187,0.526517,0.524814,0.481383,0.505102,0.529853,0.518907,1430.0,1.0
9,0.775924,0.838166,0.826102,0.873044,0.908527,0.957195,0.983428,1.024460,1.011272,1.076229,...,0.518332,0.499111,0.517247,0.508426,0.534802,0.517532,0.523042,0.494015,1420.0,1.0


In [20]:
df_mean.shape

(3276, 603)

In [21]:
df_mean.drop(0, inplace=True)    #NaN değerlerden oluşan ilk satırı attık.

In [22]:
df_mean.sort_values(by=601)     #Her kalınlıktan 25 adet * 131 = 3275 row

,0,1,2,3,4,5,6,7,8,9,...,593,594,595,596,597,598,599,600,601,602
3275,0.730400,0.768940,0.803763,0.817549,0.867651,0.869369,0.907571,0.954615,0.965236,0.999758,...,0.551087,0.554501,0.544048,0.537476,0.553187,0.546401,0.560433,0.541451,200.0,25.0
131,0.715137,0.767162,0.787861,0.785534,0.830228,0.875304,0.916727,0.964578,0.970791,1.044645,...,0.531671,0.549899,0.533478,0.524631,0.550116,0.538479,0.541174,0.554225,200.0,1.0
3013,0.698014,0.756961,0.811701,0.822029,0.806195,0.931606,0.974046,0.950033,0.964843,0.943612,...,0.528223,0.541705,0.563454,0.558754,0.550762,0.542816,0.537994,0.565332,200.0,23.0
262,0.772598,0.783998,0.856261,0.858850,0.865232,0.919448,0.963232,0.983815,1.022767,1.006694,...,0.558545,0.556152,0.551112,0.553388,0.551825,0.553293,0.563481,0.587840,200.0,2.0
2882,0.730193,0.750770,0.804687,0.833304,0.907999,0.872857,0.910761,0.958483,1.006141,1.037428,...,0.545235,0.557587,0.557157,0.548070,0.541868,0.574729,0.545722,0.549494,200.0,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,0.720714,0.760713,0.743164,0.845855,0.875793,0.937316,0.977545,0.962293,1.044815,0.995379,...,0.532385,0.523568,0.508915,0.516274,0.532927,0.531032,0.515825,0.512740,1500.0,4.0
1180,0.685663,0.797950,0.855469,0.839947,0.856990,0.914201,0.978942,1.016473,1.026928,1.045903,...,0.543427,0.533444,0.514230,0.534510,0.511246,0.533094,0.519153,0.538078,1500.0,10.0
2097,0.706736,0.753464,0.742722,0.851389,0.872106,0.877776,0.977223,0.935388,0.999377,1.026503,...,0.532780,0.527460,0.529355,0.532623,0.519785,0.486599,0.530285,0.529951,1500.0,17.0
1311,0.694735,0.741033,0.805452,0.858673,0.857954,0.949821,0.933443,0.998776,1.005560,1.000451,...,0.522563,0.539252,0.534124,0.542598,0.525955,0.515235,0.522018,0.518276,1500.0,11.0


In [23]:
df_mean.describe().T

,count,mean,std,min,25%,50%,75%,max
0,3275.0,0.788568,0.056920,0.635987,0.743723,0.789940,0.834024,0.937307
1,3275.0,0.822137,0.055600,0.656607,0.778253,0.823728,0.866343,0.977822
2,3275.0,0.854240,0.054896,0.680713,0.810190,0.857164,0.897307,0.998154
3,3275.0,0.888209,0.053322,0.734993,0.846187,0.891250,0.929611,1.039553
4,3275.0,0.919420,0.053411,0.762572,0.876001,0.922019,0.961846,1.058856
...,...,...,...,...,...,...,...,...
598,3275.0,0.480800,0.030699,0.417454,0.455830,0.474813,0.502933,0.578429
599,3275.0,0.481497,0.030872,0.405145,0.456684,0.475401,0.504781,0.577561
600,3275.0,0.481729,0.030799,0.414199,0.457186,0.476317,0.504228,0.587840
601,3275.0,850.000000,378.211155,200.000000,520.000000,850.000000,1180.000000,1500.000000


In [24]:
X = df_mean.drop(601, axis=1)     #Features

y = df_mean[601]                  #Target

print(X)
print(y)

           0         1         2         3         4         5         6    \
1     0.698106  0.763157  0.776748  0.832544  0.914430  0.916932  0.944375   
2     0.729856  0.751451  0.827925  0.801372  0.852115  0.927645  0.951585   
3     0.669586  0.733506  0.868904  0.841791  0.832750  0.914250  0.933147   
4     0.762774  0.802765  0.803625  0.847220  0.929512  0.954482  0.998781   
5     0.714084  0.810382  0.775471  0.806111  0.873733  0.946925  0.942013   
...        ...       ...       ...       ...       ...       ...       ...   
3271  0.680470  0.762537  0.771428  0.785900  0.841452  0.882739  0.975252   
3272  0.693063  0.770258  0.797888  0.813125  0.829096  0.882887  0.904326   
3273  0.726921  0.710240  0.782002  0.792297  0.863234  0.856106  0.926991   
3274  0.701827  0.763658  0.736724  0.833147  0.810806  0.913451  0.917418   
3275  0.730400  0.768940  0.803763  0.817549  0.867651  0.869369  0.907571   

           7         8         9    ...       592       593    

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=44)

In [27]:
#X_train = X[X[602] < 20] 
#X_train = X_train.drop(602, axis=1)

In [28]:
#X_test = X[X[602] > 20] 
#X_test = X_test.drop(602, axis=1)

In [29]:
#y_train = X[X[602] < 20] 
#y_train = y_train.drop(602, axis=1)

In [30]:
#y_test = X[X[602] > 20] 
#y_test = y_test.drop(602, axis=1)

In [31]:
from sklearn.linear_model import LinearRegression

In [32]:
model = LinearRegression() 

In [33]:
model.fit(X_train, y_train)

LinearRegression()

In [34]:
y_pred = model.predict(X_test)

In [35]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [36]:
r2_score = r2_score(y_test, y_pred)
r2_score

0.9993434853085279

In [37]:
mae = mean_absolute_error(y_test, y_pred)
mae

7.81672578538526

In [38]:
mse = mean_squared_error(y_test, y_pred)
mse

95.49945568514559

In [39]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse

9.772382293235646

In [40]:
def adj_r2(y_test, y_pred, df_mean):
    n = df_mean.shape[0]   
    p = df_mean.shape[1]-1
    adj_r2 = 1 - (1-r2_score)*(n-1)/(n-p-1)
    return adj_r2

In [41]:
adj_r2(y_test, y_pred, df_mean)

0.9991955729416618

In [42]:
from sklearn.model_selection import cross_validate, cross_val_score 

In [43]:
model = LinearRegression()
scores = cross_validate(model, X_train, y_train, scoring= ["r2", "neg_mean_absolute_error","neg_mean_squared_error", "neg_root_mean_squared_error"], cv = 15)

In [44]:
pd.DataFrame (scores, index = range(1, 16))

,fit_time,score_time,test_r2,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_root_mean_squared_error
1,0.341802,0.006997,0.999335,-7.980405,-100.501375,-10.025037
2,0.336908,0.007995,0.999244,-7.930538,-99.932270,-9.996613
3,0.345606,0.006997,0.999221,-7.979206,-109.383726,-10.458668
4,0.378768,0.008010,0.999305,-7.891234,-97.976913,-9.898329
5,0.339454,0.005998,0.999312,-8.070410,-103.947997,-10.195489
6,0.341159,0.006011,0.999279,-8.099534,-98.700186,-9.934797
7,0.345370,0.005997,0.999304,-7.803674,-99.550717,-9.977511
8,0.334047,0.007010,0.999412,-7.532612,-86.900721,-9.322056
9,0.334119,0.007010,0.999141,-8.555527,-110.691580,-10.521007
10,0.332716,0.005996,0.999425,-6.865793,-80.444347,-8.969077


In [46]:
scores = pd.DataFrame(scores, index = range(1,16))    #Cross Validation sonuçlarının ortalaması
scores.iloc[:, 2:].mean()

test_r2                               0.999286
test_neg_mean_absolute_error         -7.927029
test_neg_mean_squared_error        -100.414424
test_neg_root_mean_squared_error    -10.004353
dtype: float64